# 3장. 다항 선형 회귀와 정규화 기법

---

## 학습 목표
- 정규화 기법이 왜 필요한지를 이해합니다.
- 다항 선형 회귀에서 사용되는 릿지, 라쏘, 엘라스틱 정규화 기법에 대해서 학습합니다.

---

## 목차

### 1. 정규화 기법 개념
1. 선형 회귀의 정규화 기법

### 2. 릿지 정규화 기법
1. Loss 함수와 규제항
2. 규제항 하이퍼 파라미터

### 3. 라쏘, 엘라스틱 넷 정규화 기법
1. 라쏘 정규화
2. 엘라스틱 넷 정규화


---

## 2. 릿지 정규화 기법

봉우리를 만들 수 있는 고차항의 선형 파라미터를 줄이기 위해서는 학습용 데이터에 너무 정확하게는 맞지 않게 학습시켜야 합니다.

선뜻 이해하기 어렵겠지만 학습용 데이터에 완벽히 맞을 수록 모델의 형태는 많이 휘어지게 됩니다.

그렇다면 학습용 데이터를 학습할 때, 정확도를 떨어트려야 합니만 어느정도 어떤 방법을 떨어 트려야 하는지가 중요합니다.

릿지(Ridge) 정규화 방식은 loss 함수에 페널티를 주는 방식으로 학습 시의 정확도를 제어합니다.

### 2-1. Loss 함수와 규제항

선형 회귀에서의 Loss 함수는 MSE로 정의 되었습니다. 따라서 MSE 값이 줄어들게 되면 Loss 값이 줄어드는 관계를 갖게 됩니다.

##### loss 함수 정의

> $$Loss(\mathbf{w}) = MSE(\mathbf{w})\;\;\;\; \mathbf{w}=\begin{pmatrix}
w_0 \\ 
w_1 \\ 
w_2 \\ 
\vdots \\ 
w_p
\end{pmatrix}$$

$p+1$개의 파라미터가 존재하는 선형 회귀 모델을 가정합니다.  

여기에 정규항을 추가하게 되면 아래의 수식과 같습니다.

##### 정규항 추가 loss 함수

> $$Loss(\mathbf{w}) = MSE(\mathbf{w})+\frac{\lambda}{N} \sum_{i=0}^{p}w_{i}^{2}$$

규제항이 추가 되면서 어떤 변화가 있는지 알아봅시다.

loss 함수를 줄이는 방법을 사용해서 선형 파라미터를 조절하여 MSE 값을 줄인다고 생각해봅시다.

선형 파라미터가 조정되어 MSE 값은 줄어들겠지만 규제항은 특정 값을 갖게 됩니다.

따라서 MSE 값을 줄이는 방향만으로 선형 파라미터를 조정하게 된다면 규제항이 커질 수 있는 부분이 있기에 MSE 최적화와 규제항 간의 **trade-off**가 존재합니다.

#### 3차항 함수 모델 예

3차 함수 모델에 대해서 예를 들어 봅시다.

MSE를 최소로 만들 수 있는 선형 파라미터는 $w_0=1$, $w_1=2$, $w_2=4$, $w_3=3$이고 이때의 MSE값은 20으로 가정합시다.

$\lambda=4$이라 가정한다면 규제항의 값은 $1+2^{2}+4^{2}+3^{2}=30$이 됩니다.

따라서 Loss 값은 20+30=50이 됩니다.

위 값을 보면 알수 있듯이 MSE 값에 비해서 규제항의 값이 큽니다. 만약 $w_2=3$으로 변경했을 때, MSE값이 22가 되었다면 규제항의 값은 23이 됩니다. 따라서 Loss 값은 45로 더 줄어든 형태가 됩니다.

이처럼 선형 파라미터의 절대 값을 줄이는 것으로 규제항을 줄일 수 있고 그 정도가 MSE 증가보다 크다면 MSE 값을 증가하는 선형 파라미터를 선택하게 됩니다.

하지만 $w_2=2$ 설정하였을때 MSE 값이 30으로 증가했다면 규제항 값 18로 loss 값이 48이 됩니다.

이 경우에는 MSE 증가량이 규제항의 감소량 보다 많았기에 이러한 선형 파라미터는 선택이 되지 않습니다.

#### 규제항과 MSE 그래프 예

위 예시처럼 선형 파라미터 값의 조정으로 인한 MSE 증가량과 규제항 감소량 간의 **trade-off**가 존재함을 알 수 있었습니다. 

MSE와 규제항 간의 관계를 이해하기 위해서 그래프 형태로 살펴 봅시다.

먼저 1차 함수에서의 선형 파라미터 $w_0, w_1$에 따른 MSE가 아래와 같이 그려진다 가정해봅시다.

<img src="img/3-2-1.png" width="40%" height="40%" title="rid1" alt="rid1"></img>

붉은 점은 MSE 값이 최소가 되는 지점이고 멀리 떨어질수록 MSE 값이 증가함을 알 수 있습니다.

<img src="img/3-2-2.png" width="40%" height="40%" title="rid2" alt="rid2"></img>

규제항의 경우 원점을 기준으로 원형 분포로 값이 증가함을 알 수 있습니다.

이는 2차 항의 합이 원형 함수로 표현되기 때문입니다.

만약 MSE 값 최소 지점이 0이라면 이 때의 선형 파라미터에 대한 Loss 값을 계산하면 아래 그림과 같이 0+50=50으로 계산됩니다.

<img src="img/3-2-3.png" width="40%" height="40%" title="rid3" alt="rid3"></img>

선형 파라미터를 조절하여 MSE 값을 10으로 증가 시켰다면 아래 그림과 같이 규제항의 반지름이 줄어들게 되며 35란 값을 갖게 됩니다.

<img src="img/3-2-4.png" width="40%" height="40%" title="rid4" alt="rid4"></img>

같은 방식으로 MSE 값을 10으로 증가시키지만 $w_0, w_1$를 아래와 같이 변경하면 규제항의 크기는 오히려 늘어나게 됩니다.

<img src="img/3-2-5.png" width="40%" height="40%" title="rid5" alt="rid5"></img>

따라서 원점에서 시작되는 원의 반지름의 크기의 제곱과 MSE의 크기의 합이 최소가 되는 지점을 찾는 것이 릿지 정규화 과정이라 할 수 있습니다.

---

**심화 학습 - 릿지 정규화 최적화**

**Least square**

loss 함수가 바뀌었다면 least square 기법은 그대로 쓸 수 있을까요?

아래 수식을 따라가며 릿지 정규화 기법에서의 least square solution을 구해보겠습니다.

행렬을 사용하여 loss 함수를 다음과 같이 정의할 수 있습니다.

##### 릿지 loss 함수

> $$Loss(\mathbf{w})=\frac{1}{N}(\mathbf{y}-X\mathbf{w})^{T}(\mathbf{y}-X\mathbf{w})+\frac{\lambda}{N} \mathbf{w}^{T}\mathbf{w}$$

기울기가 0인 지점을 찾아야 하기에 $\mathbf{w}$으로 편미분을 합니다.

##### loss 함수 편미분

> $$\frac{\partial Loss}{\partial \mathbf{w}}=\frac{1}{N}(\frac{\partial \mathbf{y}^{T}\mathbf{y}}{\partial \mathbf{w}} - \frac{\partial \mathbf{y}^{T}X\mathbf{w}}{\partial \mathbf{w}} - \frac{\partial \mathbf{w}^{T}X^{T}\mathbf{y}}{\partial \mathbf{w}} + \frac{\partial \mathbf{w}^{T}X^{T}X\mathbf{w}}{\partial \mathbf{w}}+\frac{\partial \lambda \mathbf{w}^{T}\mathbf{w}}{\partial \mathbf{w}})$$

> $$\frac{\partial Loss}{\partial \mathbf{w}}=\frac{1}{N}(0-\mathbf{y}^{T}X-(X^{T}\mathbf{y})^{T}+2\mathbf{w}^{T}X^{T}X+2\lambda \mathbf{w}^{T})$$

기울기가 0인 지점을 찾기 위하여 편미분 한 것이므로 다음과 같은 방정식을 만족하는 $\mathbf{w}$는 $\mathbf{\widehat{w}}$입니다.

##### 최적의 w 값을 찾는 과정

> $$\begin{aligned}
\frac{\partial Loss}{\partial \mathbf{\widehat{w}}}&=0\\
\frac{1}{N}(-2\mathbf{y}^{T}X+2\mathbf{w}^{T}X^{T}X+2\lambda \mathbf{w}^{T})&=0\\
\mathbf{\widehat{w}}^{T}(X^{T}X+\lambda I) &= \mathbf{y}^{T}X\\
(X^{T}X+\lambda I)\mathbf{\widehat{w}} &= X\mathbf{y}
\end{aligned}$$

여기서 $(X^{T}X+\lambda I)$의 역행렬이 존재한다 가정한다면, $\widehat{w}$는 다음과 같이 정리할 수 있습니다.

##### 최적의 해

> $$\mathbf{\widehat{w}} = (X^{T}X+\lambda I)^{-1}X\mathbf{y}$$

결과적으로 원래 solution에서 $\lambda$에 의해 바뀌어진 solution이 구해집니다. 

**Gradient descent**

릿지 정규화도 gradient descent 방식으로 최소의 Loss 값을 찾을 수 있습니다.

규제항이 붙었지만 2차항이기에 local minima를 유발하지 않고 기존 고차항 선형 회귀와 같이 loss값을 편미분하여 gradient를 구할 수 있습니다.

##### gradient 계산

> $$gradient= \triangledown Loss = \begin{pmatrix}
\frac{\partial Loss}{\partial w_0} \\ 
\frac{\partial Loss}{\partial w_1} \\ 
\vdots \\ 
\frac{\partial Loss}{\partial w_p}
\end{pmatrix}$$

##### loss 함수 w_i에 대한 편미분 

> $$\frac{\partial Loss(\mathbf{w})}{\partial w_i} = \frac{\partial MSE(\mathbf{w})}{\partial w_i}+\frac{2\lambda}{N} w_{i}$$

편미분된 식을 보면 알 수 있듯이, 기존 MSE를 편미분한 $\lambda$ 만큼 엇나간 방향으로 바뀌게 됩니다.

이렇게 구한 gradient를 사용하여 아래와 같이 파라미터를 업데이트 할 수 있습니다.

##### w_i 업데이트 수식

> $$\mathbf{w}^{t+1} = \mathbf{w}^{t} - \alpha \triangledown Loss(\mathbf{w})$$

---

### 2-2. 규제항 하이퍼 파라미터

규제항에는 하이퍼 파라미터로 $\lambda$가 곱해집니다. 이것은 규제항의 조절하는 파라미터로 얼마나 크게 규제를 할 것인가를 의미합니다. 

위 예제는 $\lambda=1$인 경우이기에 규제항의 크기는 원의 반지름 크기의 제곱의 값이였지만 $\lambda$ 값이 변하면 그 크기의 배수만큼 크기도 변화합니다.

따라서 $\lambda$ 값이 커지게 되면 기존의 반지름 제곱보다 큰 규제항을 갖게 됨으로 규제항의 의미가 커지게 됩니다.

반대로 $\lambda$ 값이 작아지면 그만큼 규제항의 변화가 작아져 규제항의 의미가 작아집니다.

일반적으로 $\lambda=1$을 두고 시작하지만 정규화 된 그래프를 보고 규제를 주고 싶다면 $\lambda$ 값을 올리는 방향으로, 규제를 풀고 싶다면 $\lambda$ 값을 내리는 방향으로 최적화를 합니다.

**scikit-learn을 사용한 릿지 정규화**

scikit-learn에서는 릿지 정규화는 `Ridge` class를 사용하여 수행할 수 있습니다.

`Ridge` class는 기존에 사용하던 `LinearRegression` class와 거의 흡사합니다.

`Ridge` class는 초기화 시, $\lambda$ 값을 의미하는 `alpha`값을 설정하여야 합니다.

##### <예제 1> 릿지 정규화

`2-2. <예제 4>` 에서와 같은 데이터를 사용하여 릿지 정규화를 수행하고 loss 값을 출력해 봅시다.

In [3]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

# 학습용 데이터
feature_data = np.array([1, 2, 3, 4]).reshape((-1,1))
label_data = np.array([3.1,4.9,7.2,8.9]).reshape((-1,1))

# 4차항 선형 회귀에 해당되는 4를 초기값으로 입력
poly = PolynomialFeatures(4)

# fit 함수를 사용하여 X로 변환 작업을 수행
X = poly.fit_transform(feature_data)
print("X: \n{}\n".format(X))

# 릿지 모델 설정 알파 값을 바꿔가며 수행
ridge_model = Ridge(alpha=1)

# 학습 수행
ridge_model.fit(X, label_data)

# scikit-learn 에서는 loss 함수가 모델안에 내정되어 있지 않기에 정의
def loss(prediction, label):
        
    error = label - prediction
    ls = np.mean(error**2)

    return ls

print("loss: {}\n".format(loss(ridge_model.predict(X), label_data))) 

X: 
[[  1.   1.   1.   1.   1.]
 [  1.   2.   4.   8.  16.]
 [  1.   3.   9.  27.  81.]
 [  1.   4.  16.  64. 256.]]

loss: 0.00503462137839671



위 실습에서 모델의 $\lambda$ 값을 의미하는 `alpha`값을 바꿔가며 loss 값이 어떻게 바뀌는지 수행해 봅시다.

---